In [ ]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [ ]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# parameters
country = "united-kingdomfa"
city = "scotland"
street = "edinburgh-salamander-st"
aqicn_url = "https://api.waqi.info/feed/@5986"
sensor_secret_name = f"SENSOR_LOCATION_JSON_{street.replace(' ', '_')}"

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [ ]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret(sensor_secret_name).value
location = json.loads(location_str)

#country=location['country']
#city=location['city']
#street=location['street']
#aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [ ]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [ ]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

In [ ]:
aq_today_df.info()

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [ ]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

In [ ]:
daily_df.info()

In [ ]:
# for c level implementation, read the new lag features from hopswork
# Get recent PM2.5 history to compute lag features (last 3 days)
# We need to read from the feature group to get historical PM2.5 values
recent_days = 3
cutoff_date = today - datetime.timedelta(days=recent_days + 1)

# Save original aq_today_df with all columns (including url) before processing
aq_today_df_original = aq_today_df.copy()

# Read recent air quality data for this street
recent_aq = air_quality_fg.filter(
    (air_quality_fg.street == street) & 
    (air_quality_fg.date >= cutoff_date)
).read()

# Sort by date
recent_aq = recent_aq.sort_values('date')

# Combine recent data with today's data for lag computation
# Include url column if it exists in recent_aq, otherwise we'll add it back later
cols_to_merge = ['date', 'pm25', 'country', 'city', 'street']
if 'url' in recent_aq.columns:
    cols_to_merge.append('url')
    
aq_with_history = pd.concat([recent_aq[cols_to_merge], 
                             aq_today_df[cols_to_merge]], 
                            ignore_index=True)
aq_with_history = aq_with_history.sort_values('date').reset_index(drop=True)

# Compute lag features
aq_with_history['pm25_lag_1'] = aq_with_history['pm25'].shift(1)
aq_with_history['pm25_lag_2'] = aq_with_history['pm25'].shift(2)
aq_with_history['pm25_lag_3'] = aq_with_history['pm25'].shift(3)

# Compute rolling 3-day mean
aq_with_history['pm25_roll_3'] = aq_with_history['pm25'].rolling(window=3, min_periods=1).mean()

# Extract only today's row with lag features
aq_today_df = aq_with_history[aq_with_history['date'] == pd.to_datetime(today)].copy()

# Ensure url column is present (add from original if missing)
if 'url' not in aq_today_df.columns and 'url' in aq_today_df_original.columns:
    aq_today_df['url'] = aq_today_df_original['url'].iloc[0]

# Fill NaN values with 0.0 (for the first few days when history is insufficient)
aq_today_df[['pm25_lag_1', 'pm25_lag_2', 'pm25_lag_3', 'pm25_roll_3']] = \
    aq_today_df[['pm25_lag_1', 'pm25_lag_2', 'pm25_lag_3', 'pm25_roll_3']].fillna(0.0)

# Ensure date is datetime
aq_today_df['date'] = pd.to_datetime(aq_today_df['date'])

print("Today's data with lag features:")
print(aq_today_df[['date', 'pm25', 'pm25_lag_1', 'pm25_lag_2', 'pm25_lag_3', 'pm25_roll_3']])
print(f"\nColumns in aq_today_df: {aq_today_df.columns.tolist()}")
aq_today_df


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
# Insert new data
air_quality_fg.insert(aq_today_df)

In [ ]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
